In [78]:
import sys
sys.path.append('/Users/sdkolak/anaconda3/envs/hector/lib/python3.9/site-packages')

In [79]:
import os 
import json

In [80]:
prefix_only_path = "../data/rep/no_gap_bleu_score/p"
prefix_suffix_path = "../data/rep/no_gap_bleu_score/ps"

In [81]:
def get_files(path):
    files = []
    for file in os.listdir(path):
        files.append(os.path.join(path, file))
    return files

In [82]:
prefix_only = get_files(prefix_only_path)
prefix_suffix = get_files(prefix_suffix_path)

In [83]:
def get_data(files, context):
    perf_match_count = 0
    total = 0
    matched = []
    for file in files:
        proj, bug = file.split("/")[-1].replace(".json", "").split("_")
        proj_bug = (proj,bug)
        f = open(file, "r")
        bs_sum, count = 0,0
        for line in f:
            data = json.loads(line)
            rank = data["rank"]
            bleu_score = float(data["bleu_score"])
            if bleu_score == 1.0 and proj_bug not in matched: 
                perf_match_count += 1 
                matched.append(proj_bug)
            bs_sum += bleu_score 
            count += 1
        total += 1
    print("average bleu score:", (bs_sum/count))
    print("number of perf matches:", perf_match_count, "total:", total)
    print("perf match percent:", (perf_match_count/total))
    return matched 

In [84]:
print("prefix only:")
pre_perf = get_data(prefix_only, 'p')
print("with suffix:")
pre_suf_perf = get_data(prefix_suffix, 'ps')

#pre_set = set(pre_perf)
#pre_suf_set = set(pre_suf_perf)

#print(len(pre_suf_set - pre_set))

#for file in pre_perf: 
#    if file not in pre_suf_perf:
        #print("new", file)
#    if file in pre_suf_perf:
#        print("same", file)

prefix only:
average bleu score: 0.7734974253338421
number of perf matches: 19 total: 95
perf match percent: 0.2
with suffix:
average bleu score: 0.8322640918359336
number of perf matches: 30 total: 95
perf match percent: 0.3157894736842105


In [122]:
def get_data(files, context):
    perf_match_count = 0
    total = 0
    matched = []
    rank_bs = {}
    count = 0
    for file in files:
        proj, bug = file.split("/")[-1].replace(".json", "").split("_")
        proj_bug = (proj,bug)
        f = open(file, "r")
        bs_sum = 0
        for line in f:
            data = json.loads(line)
            rank = data["rank"]
            bleu_score = float(data["bleu_score"])
            if rank not in rank_bs:
                rank_bs[rank] = []
            rank_bs[rank].append(bleu_score)
            if bleu_score == 1.0 and proj_bug not in matched: 
                perf_match_count += 1 
                matched.append(proj_bug)
            bs_sum += bleu_score 
            count += 1
        total += 1
    #print("average bleu score:", (bs_sum/count))
    #print("number of perf matches:", perf_match_count, "total:", total)
    #print("perf match percent:", (perf_match_count/total))
    return rank_bs

In [123]:
def avrg(lst):
    return sum(lst) / len(lst)

def get_avrg(rank_bs, has_suffix, data_f):
    all_bs = []
    for rank,scores in rank_bs.items():
        av = avrg(scores)
        data_f.write(str(rank) + "," + str(av) + "," + str(has_suffix)+'\n')
        all_bs.append(av)
    print("overall av:", avrg(all_bs))
        

In [124]:
data_f = open("bs_data.csv", "w+")
data_f.write("rank,avg_bs,has_suffix\n")
print("prefix only:")
p_rank_bs = get_data(prefix_only, 'p')
get_avrg(p_rank_bs, False, data_f)
print("with suffix:")
ps_rank_bs = get_data(prefix_suffix, 'ps')
get_avrg(ps_rank_bs, True, data_f)
data_f.close()

prefix only:
overall av: 0.28664165252131124
with suffix:
overall av: 0.35965025073825774
